In [1]:
%load_ext autoreload

In [2]:
import os

import numpy as np

from libertem import api
from libertem.udf.sum import SumUDF
from libertem.udf.base import UDF

c:\users\weber\documents\libertem\libertem\src\libertem\api.py:17: RuntimeWarning: ImportError('CuPy is not correctly installed.\n\nIf you are using wheel distribution (cupy-cudaXX), make sure that the version of CuPy you installed matches with the version of CUDA on your host.\nAlso, confirm that only one CuPy package is installed:\n  $ pip freeze\n\nIf you are building CuPy from source, please check your environment, uninstall CuPy and reinstall it with:\n  $ pip install cupy --no-cache-dir -vvvv\n\nCheck the Installation Guide for details:\n  https://docs-cupy.chainer.org/en/latest/install.html\n\noriginal error: DLL load failed: The specified module could not be found.')
  warnings.warn(repr(e), RuntimeWarning)


In [3]:
%autoreload
ctx = api.Context()

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://134.94.7.199:59935
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.7.199:59937'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.7.199:59938'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.7.199:59939'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.7.199:59940'
distributed.scheduler - INFO - Register tcp://134.94.7.199:59957
distributed.scheduler - INFO - Starting worker compute stream, tcp://134.94.7.199:59957
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://134.94.7.199:59959
distributed.scheduler - INFO - Starting worker compute stream, tcp://134.94.7.199:59959
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://134.94.7.199:59961
distributed.scheduler - INFO - Starting worker compute stream, tcp://134.94.7.19

In [4]:
ctx.executor.client.scheduler_info()

{'type': 'Scheduler',
 'id': 'Scheduler-b5b949e5-0ec4-4298-830c-a7712aec33b6',
 'address': 'tcp://134.94.7.199:59935',
 'services': {},
 'workers': {'tcp://134.94.7.199:59957': {'type': 'Worker',
   'id': 'service-0',
   'host': '134.94.7.199',
   'resources': {},
   'local_directory': 'C:\\Users\\weber\\Documents\\LiberTEM\\LiberTEM\\prototypes\\cupy\\dask-worker-space\\worker-ljxzyamy',
   'name': 'service-0',
   'nthreads': 1,
   'memory_limit': 2065603584,
   'last_seen': 1588015974.8182626,
   'services': {},
   'metrics': {'cpu': 0.0,
    'memory': 36147200,
    'time': 1588015974.748206,
    'read_bytes': 0.0,
    'write_bytes': 0.0,
    'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}},
   'nanny': 'tcp://134.94.7.199:59939'},
  'tcp://134.94.7.199:59959': {'type': 'Worker',
   'id': 'cuda-0',
   'host': '134.94.7.199',
   'resources': {'CUDA': 1, 'compute': 1},
   'local_directory': 'C:\\U

In [5]:
def info():
    import os
    return (os.environ.get("LIBERTEM_USE_CPU", None), os.environ.get("LIBERTEM_USE_CUDA", None))

In [6]:
ctx.executor.client.run(info)

{'tcp://134.94.7.199:59957': (None, None),
 'tcp://134.94.7.199:59959': (None, '0'),
 'tcp://134.94.7.199:59961': ('1', None),
 'tcp://134.94.7.199:59963': ('0', None)}

In [7]:
data = np.ones((4, 4, 4, 4))

ds = ctx.load("memory", data=data, num_partitions=8)

In [8]:
class DebugUDF(UDF):
    def get_result_buffers(self):
        return {
            'debug': self.buffer(kind="single", dtype="object"),
            'on_device': self.buffer(kind="sig", dtype=np.float32, where="device"),
            'backend': self.buffer(kind="nav", dtype="object"),
        }
    
    def preprocess(self):
        self.results.debug[0] = dict()
    
    def process_partition(self, partition):
        cpu = os.environ.get("LIBERTEM_USE_CPU", None)
        cuda = os.environ.get("LIBERTEM_USE_CUDA", None)
        self.results.debug[0][self.meta.slice] = (cpu, cuda)
        self.results.on_device[:] += self.xp.sum(partition, axis=0)
        self.results.backend[:] = self.meta.backend
        print(f"meta backend {self.meta.backend}")
        
    def merge(self, dest, src):
        de, sr = dest['debug'][0], src['debug'][0]
        for key, value in sr.items():
            assert key not in de
            de[key] = value
            
        dest['on_device'][:] += src['on_device']
        dest['backend'][:] = src['backend']
            
    def get_backends(self):
        return ('cupy', 'numpy')
        

In [9]:
res = ctx.run_udf(udf=DebugUDF(), dataset=ds)
print(res['debug'].data)
print(res['on_device'].data)
print(res['backend'].data)

UDFMeta backend: numpy
[{<Slice origin=(4, 0, 0) shape=(2, 4, 4)>: ('0', None), <Slice origin=(10, 0, 0) shape=(2, 4, 4)>: ('0', None), <Slice origin=(0, 0, 0) shape=(2, 4, 4)>: ('1', None), <Slice origin=(6, 0, 0) shape=(2, 4, 4)>: ('1', None), <Slice origin=(12, 0, 0) shape=(2, 4, 4)>: ('1', None), <Slice origin=(2, 0, 0) shape=(2, 4, 4)>: (None, '0'), <Slice origin=(8, 0, 0) shape=(2, 4, 4)>: (None, '0'), <Slice origin=(14, 0, 0) shape=(2, 4, 4)>: (None, '0')}]
[[16. 16. 16. 16.]
 [16. 16. 16. 16.]
 [16. 16. 16. 16.]
 [16. 16. 16. 16.]]
[['numpy' 'numpy' 'cupy' 'cupy']
 ['numpy' 'numpy' 'numpy' 'numpy']
 ['cupy' 'cupy' 'numpy' 'numpy']
 ['numpy' 'numpy' 'cupy' 'cupy']]
